# Detection - **YOLO**

In [1]:
import cv2
import numpy as np
from pathlib import Path

In [2]:
PROJECT_ROOT = Path().resolve().parent.parent
INPUT_VIDEO_PATH = str(PROJECT_ROOT / "data" / "recording_7" / "Recording_7.mp4")
OUTPUT_VIDEO_PATH = str(PROJECT_ROOT / "data" / "recording_7" / "Recording_7_YOLO.mp4")

PATH_TO_WEIGHTS = str(PROJECT_ROOT / "YOLO" / "yolov3.weights")
PATH_TO_CFG = str(PROJECT_ROOT / "YOLO" / "yolov3.cfg")

cap = cv2.VideoCapture(INPUT_VIDEO_PATH)

# Get video properties (width, height, and FPS) for saving the output video
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Create VideoWriter object to save the new video
output_video = cv2.VideoWriter(
    OUTPUT_VIDEO_PATH, cv2.VideoWriter_fourcc(*"MP4V"), fps, (frame_width, frame_height)
)

OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


In [3]:
# Load YOLO
net = cv2.dnn.readNet(PATH_TO_WEIGHTS, PATH_TO_CFG)
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Prepare the frame for YOLO
    blob = cv2.dnn.blobFromImage(
        frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False
    )
    net.setInput(blob)
    outs = net.forward(output_layers)

    # Post-process YOLO outputs
    class_ids = []
    confidences = []
    boxes = []

    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]

            if confidence > 0.5:  # You can adjust the threshold
                center_x = int(detection[0] * frame.shape[1])
                center_y = int(detection[1] * frame.shape[0])
                w = int(detection[2] * frame.shape[1])
                h = int(detection[3] * frame.shape[0])

                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    # Use NMS (Non-Maximum Suppression) to filter overlapping boxes
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    # Draw bounding boxes around the detected bowling pins
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # Write the frame with bounding boxes to the output video
    output_video.write(frame)

    cv2.imshow("Frame", frame)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
output_video.release()

cv2.destroyAllWindows()

print("Tracking video saved successfully to:", OUTPUT_VIDEO_PATH)

Tracking video saved successfully to: /home/davic/projects/IACV_project/bowling-analysis/data/recording_7/Recording_7_YOLO.mp4
